# Imports

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
import pandas
import numpy
import data_massage
import itertools
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

# tensorflow on my laptop seems to not obtain the correct amount of memory
# the GPU on it has 2G, but asking for 1300 was too much, 1000 worked, so there you go
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_virtual_device_configuration(
          gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1000)])

In [ ]:
metadata, full_data, induction_only = data_massage.load_data()
summarized_data = data_massage.summarize_data(metadata, induction_only)

lstm_data = induction_only
features = sorted(["R_mean", "Temp.", "Humidity"])

hidden_layers = tuple(map(lambda p: (len(features) ** p), [3,4,5,4,3]))
feature_permutations = list(itertools.chain.from_iterable(map(lambda r: itertools.combinations(features, r), range(1, 1 + len(features)))))

In [ ]:
results = {}

for perm in feature_permutations:
    feature_set = list(perm)
    lstm_train_x, lstm_test_x, lstm_train_y, lstm_test_y = train_test_split(
          lstm_data[feature_set]
        , lstm_data["label"]
        , test_size=0.2
        , stratify=lstm_data["label"]
        , random_state=7
    )

    label_encoder = LabelEncoder()
    label_encoder.fit(lstm_train_y)
    label_encoder.classes_
    lstm_train_y = label_encoder.transform(lstm_train_y)
    lstm_test_y = label_encoder.transform(lstm_test_y)
    
    model = Sequential()
    model.add(LSTM(units = lstm_train_x.shape[1], return_sequences = True, input_shape=(lstm_train_x.shape[1],1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units = lstm_train_x.shape[1] ** 2, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = lstm_train_x.shape[1] ** 3, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units = lstm_train_x.shape[1]))
    model.add(Dropout(0.2))
    model.add(Flatten())
    # Adding the output layer
    model.add(Dense(units = lstm_train_x.shape[1]))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['accuracy'])

    # Fitting the RNN to the Training set
    model.fit(lstm_train_x, lstm_train_y, epochs = 30, batch_size = 4, verbose=False)
    
    model.evaluate(lstm_test_x, lstm_test_y)
    
    key = ','.join(feature_set)
    results[key] = {"evaluation": model.evaluate(lstm_test_x, lstm_test_y)}
    
    print(key)
    print(results[key]["evaluation"])
    print("\n\n\n")